# Часть 2. Универсальные функции

## 2.1. Почему циклы медленные

В прошлой части была рассмотрена одна из причин, почему скорость нативных объектов в Python ниже. Перейдём теперь к обсуждению производительности (а точнее медлительности) циклов.

Напишем две реализации простой функции, которая возводит все числа массива в квадрат:
- одну с использованием циклов
- вторую с помощью универсальных функций (о них детальнее расскажем позже)

In [3]:
import numpy as np

In [4]:
def square_elements_loop(array_object):
    output = np.empty(len(array_object))
    for i, x in enumerate(array_object):
        output[i] = x * x
    return output

def square_elements_ufuncs(array_object):
    return array_object * array_object

Проверим, что обе функции выдают одинаковый результат:

In [5]:
x = np.arange(1, 11, dtype="float")
print(square_elements_loop(x))
print(square_elements_ufuncs(x))

[  1.   4.   9.  16.  25.  36.  49.  64.  81. 100.]
[  1.   4.   9.  16.  25.  36.  49.  64.  81. 100.]


Далее посмотрим, сколько времени требуется на выполнение это функции на достаточно большом массиве:

In [6]:
x_big = np.random.random(10_000_000)

In [7]:
%%timeit
square_elements_loop(x_big)

2.19 s ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
square_elements_ufuncs(x_big)

32.9 ms ± 5.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


По результатам выше очевидно, что на порядки выгоднее использовать именно подход без циклов. Именно поэтому механизм UFuncs позволяет максимально эффективно

## 2.2. UFuncs - что это такое

Если говорить об использовании циклов, то иначе их можно навать "поэлементными" операциями. В совокупности с механизмами, необходимыми для реализации динамической типизации, overhead'ы, которые ухудшают производительность на каждом шаге цикла, это приводит удручающим последствиям.

UFuncs (или Universal Functions) реализуют так называем векторизированный подход, когда повторяющиеся операции над элементами выполняются оптимизированным способом.

Чаще всего используются два вида универсальных функций:
- унарные
- бинарные

Первые применяются к одному массиву, последний требуют, очевидно, пару. Рассмотрим несколько примеров ниже

In [9]:
x = np.arange(1, 6, dtype="float")
x

array([1., 2., 3., 4., 5.])

In [10]:
print("x + 7 =", x + 7)
print("x - 3 =", x - 3)
print("x * 5 =", x * 5)
print("x / 2 =", x / 2)

x + 7 = [ 8.  9. 10. 11. 12.]
x - 3 = [-2. -1.  0.  1.  2.]
x * 5 = [ 5. 10. 15. 20. 25.]
x / 2 = [0.5 1.  1.5 2.  2.5]


На базовой арифметике применение UFuncs не ограничиваются, есть реализации для:
- тригонометрических функций
- экспонента и логарифмы
- специализированные функции из высшей математики
- и многие другие

## 2.3. Дополнительные возможности UFunc

Порой удобно проводить операции inplace (сразу записывать результаты применения функции в существующую переменную):

In [11]:
x = np.arange(1, 6)
y = np.zeros(len(x), dtype=int)
x, y

(array([1, 2, 3, 4, 5]), array([0, 0, 0, 0, 0]))

In [12]:
# выполним умножение массива x на 10 и сразу запишем результат в y
np.multiply(x, 10, out=y)
print(y)

[10 20 30 40 50]


In [13]:
# аналогично можно сделать с одной переменной
np.power(2, x, out=x)
print(x)
# данный синтаксис эквивалентен x = x ** 2

[ 2  4  8 16 32]


Также следует отметить, что комбинирование с концепцией no-copy view можно записывать результат применения UFuncs в отдельные части массива

In [14]:
x = np.arange(1, 6)
y = np.zeros(10, dtype=int)
x, y

(array([1, 2, 3, 4, 5]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [15]:
# умножим каждый из элементов массива x на 2 и запишем на чётные позиции в массив y
np.multiply(x, 2, out=y[1::2])
y

array([ 0,  2,  0,  4,  0,  6,  0,  8,  0, 10])

# Appendix

NumPy Data types: https://numpy.org/doc/stable/reference/arrays.dtypes.html

NumPy UFuncs: https://numpy.org/doc/stable/reference/ufuncs.html